# Carga de Datos

Hacemos distincción para posible estudio de rendimientos en la forma de guardado en disco y su lectura.

#### Carga Desde Binarios

In [ ]:
\ts historical: get `:Data/DataWarehouse/Tables/historical

In [ ]:
\ts resume_etf: get `:Data/DataWarehouse/Tables/resume_etf

#### Carga Desde Tablas Splayed

In [ ]:
\ts load `:Data/DataWarehouse/Tables/historicalSplayed/sym;
\ts historicalS: get `:Data/DataWarehouse/Tables/historicalSplayed/historical

In [ ]:
\ts load `:Data/DataWarehouse/Tables/resume_etfSplayed/sym;
\ts resume_etfS: get `:Data/DataWarehouse/Tables/resume_etfSplayed/resume_etf

# Cálculos

# Visualización de Datos

Para este apartado usaremos las sinergias entre KDB+/Q y Python, que al usar Jupyter Notebook se nos facilita aún más esa integración.

Usaremos el módulo y biblioteca por excelencia para esta representación gráfica en Python, [Matplotlib](https://matplotlib.org/stable/).

En primer lugar cargaremos en las variables **plt** y **mdates** los módulos de Python más usados para la representación gráfica y manipulación de fechas.

In [ ]:
\l p.q
plt:.p.import[`matplotlib;`:pyplot]
mdates:.p.import[`matplotlib;`:dates]
system "pip install -r requirements.txt"

In [ ]:
etfs: exec ticker from resume_etf

In [ ]:
vol_x:{[ETF]
    string each (select date from historical where ticker=ETF, daily_retns <>0N)`date
 }
vol_y:{[ETF]
    exec daily_retns from historical where ticker=ETF, daily_retns<> 0N
 }


In [ ]:
count select date,daily_1yvol from historical where ticker=`EWJ, daily_1yvol < 0.1

In [ ]:
daily_return:{ [ETFa]

    plt[`:figure][(`figsize);(22;14)];
    plt[`:plot][vol_x[ETFa];vol_y[ETFa];"r+"];
    plt[`:axhline][exec varianza from res where ticker=ETFa];
    ax: plt[`:gca][];
    ax[`:xaxis][`:set_major_locator][mdates[`:AutoDateLocator][]];
    plt[`:xticks][;][;][(`rotation;45)];
    plt[`:gcf][][`:autofmt_xdate][];  
    title: "DAILY RETURN - ",(string ETFa);
    plt[`:title][title];
    plt[`:xlabel]["DATE"];
    plt[`:ylabel]["VALUE UP/DOWN"];
    plt[`:legend][("DAILY";"0")];
    plt[`:show][];
    plt[`:clf][];
 };
{daily_return[x]} each etfs;

In [ ]:
disp_vol:{ [ETFa]
    plt[`:figure][(`figsize);(22;14)];
    plt[`:plot][vol_x1y[ETFa];vol_y1y[ETFa];"r-"];
    plt[`:plot][vol_x[ETFa];vol_y[ETFa];"b+"];
    ax: plt[`:gca][];
    ax[`:xaxis][`:set_major_locator][mdates[`:AutoDateLocator][]];
    plt[`:xticks][;][;][(`rotation;45)];
    plt[`:gcf][][`:autofmt_xdate][];  
    title: "DAILY RETURN - ",(string ETFa);
    plt[`:title][title];
    plt[`:xlabel]["DATE"];
    plt[`:ylabel]["VALUE UP/DOWN"];
    plt[`:legend][("DAILY";"0")];
    plt[`:show][];
    plt[`:clf][];
 }
{disp_vol[x]} each etfs;

In [ ]:
daily_vol:{ [ETFa]

    plt[`:figure][(`figsize);(22;14)];

    plt[`:plot][vol_x1y[ETFa];vol_y1y[ETFa];"r"];
    plt[`:plot][vol_x5y[ETFa];vol_y5y[ETFa];"b"];
    plt[`:plot][ax_x[ETFa];ax_y[ETFa];"g"];

    ax: plt[`:gca][];
    ax[`:xaxis][`:set_major_locator][mdates[`:AutoDateLocator][]];
    plt[`:xticks][;][;][(`rotation;45)];
    plt[`:gcf][][`:autofmt_xdate][];  
    title: "DAILY RETURN - ",(string ETFa);
    plt[`:title][title];
    plt[`:xlabel]["DATE"];
    plt[`:ylabel]["VALUE UP/DOWN"];
    plt[`:legend][("DAILY";"0")];
    plt[`:show][];
    plt[`:clf][];
 }
{daily_vol[x]} each etfs;

In [ ]:
ax_x:{[ETF]
    string each (select date from historical where ticker=ETF)`date
 }
ax_y:{[ETF]
    (select nav_value from historical where ticker=ETF)`nav_value    
 }
vol_x5y:{[ETF]
    string each (select date from historical where ticker=ETF, daily_5yvol <>0n)`date
 }
vol_y5y:{[ETF]
    (select daily_5yvol from historical where ticker=ETF, daily_5yvol <> 0n)`daily_5yvol    
 }
vol_x1y:{[ETF]
    string each (select date from historical where ticker=ETF, daily_1yvol <>0n)`date
 }
vol_y1y:{[ETF]
    (select daily_1yvol from historical where ticker=ETF, daily_1yvol <>0n)`daily_1yvol    
 }


In [ ]:
meta historical

### Generación de Gráficos Históricos y de Variación de Volumen

In [ ]:
\l QFunctions/graphics.q
\l QFunctions/queries.q
etfs: exec ticker from resume_etf
/
{in_outflow_g_F[x;2020.01.01]} each etfs;
{historical_g_F[x;2020.01.01]} each etfs;
{in_outflow_g[x]} each etfs;
{historical_g[x]} each etfs;
\

In [ ]:
{in_outflow_g_F[x;2020.01.01]} each etfs;

In [ ]:
{historical_g_F[x;2020.01.01]} each etfs;

In [ ]:
{in_outflow_g[x]} each etfs;

In [ ]:
{historical_g[x]} each etfs;

In [ ]:
// FUNCION PARA PROBAR ELEMENTOS DE MATPLOTLIB
/
plt[`:figure][(`figsize);(22;14)]; 
plt[`:plot][vol_x[`EWA];vol_y[`EWA];"o"];
ax: plt[`:gca][];

ax[`:xaxis][`:set_major_locator][mdates[`:AutoDateLocator][]];
plt[`:xticks][;][;][(`rotation;45)];
plt[`:gcf][][`:autofmt_xdate][];  

title: "DAILY RETURN - ",(string `EWA);
plt[`:title][title];
plt[`:xlabel]["DATE"];
plt[`:ylabel]["VALUE UP/DOWN"]; 
plt[`:legend][("DAILY")];
plt[`:show][];
plt[`:clf][];
\


# Modelo de Estrategia de Inversión en ETFs

En este apartado realizaremos el soporte para la elección de ETF.

En el anterior Notebook hemos realizado todos los cálculos necesarios para ahora poder selecionar y evaluar los ETF para nuestra cartera.

La técnica de selección, ponderación, etf es motivado por mi TFG de Administración de Empresas y los motivos de elección de cada apartado está explicado en él.


## Benchmark por Categoría

Se realizará el mismo benchmark para todos los ETF de la misma manera. Para poder conseguir el *Riesgo Diversificado* el mejor resultado por categoría:
> - **Mercado** en el que opera.
> - **Región** en la que opera.
> - **Clase de Equity** del que está compuesto el ETF.

## Benchmark para Modelo de Inversión

Se dará una valoración a cada elemento de:
- Tendencia del Mercado respecto a su SMA: **?%**
- Tendencia de Valoración respecto a su mSMA: **?%**
- Volatilidad a 1 año: **?%**
- Volatilidad a 6 meses: **?%**
- Volatilidad a 3 meses: **?%**
- Correlación Inflow/Outflow con Nav_Value: **?%**
- Distancia Temporal de su máximo + % valor actual respecto a él: **?%**
- Distancia Temporal de su mínimo + % valor actual respecto a él: **?%**
- Rendimiento los 5 años: **?%**
- Rendimiento los 1 año: **?%**
- Rendimiento los 6 meses: **?%**
- Drawdown a 5 años: **?%**
- Drawdown a 1 año: **?%**
- Drawdown a 6 mesess: **?%**

In [ ]:
calc_cor_all: {
 t1: exec ticker from resume_etf;
 p1: exec nav_value by ticker from historical where in_out_flow_per <> 0N;
 io1: exec in_out_flow_per by ticker from historical where in_out_flow_per <> 0N;
 c1: {p1[x] cor io1[x]} each t1;
 flip `ticker`correAll!((t1; c1))
 }

In [ ]:
calc_cor_all[]

In [ ]:
calc_cor_F:{[D_FILT];
  t: exec ticker from resume_etf;
  p: exec nav_value by ticker from historical where in_out_flow_per <> 0N,date>D_FILT;
  out: exec in_out_flow_per by ticker from historical where in_out_flow_per <> 0N,date > D_FILT;
  c: { p[x] cor out[x] } each t;
  flip `ticker`correDate!((t ; out))
 }
calc_cor_F[2020.01.01]

In [ ]:
table:()
table: update ticker: ticker from select ticker from resume_etf;
t: exec ticker from table;

In [ ]:
p: exec nav_value by ticker from historical where in_out_flow_per <> 0N;
io: exec in_out_flow_per by ticker from historical where in_out_flow_per <> 0N;
table: update corr_all: (table lj `ticker xkey (flip `ticker`correAll!((t; {p[x] cor io[x]} each t))))`correAll from table

In [ ]:
p: exec nav_value by ticker from historical where in_out_flow_per <> 0N,date>2019.09.01;
io: exec in_out_flow_per by ticker from historical where in_out_flow_per <> 0N,date > 2019.09.01;
table: update corr_5y: (table lj `ticker xkey (flip `ticker`correDate!((t; {p[x] cor io[x]} each t))))`correDate from table

In [ ]:
table

# Extras + info futuro


In [ ]:
count select ticker from resume_etf where market = `$"Developed"
select from historical where ticker = `EWA, m200sma <> 0N
count select ticker from resume_etf where market = `Emerging

//Filtrado  mezclando las dos tablas
select from historical where ticker in exec ticker from resume_etf where market = `$"Emerging"
//Lo mismo
show a: select ticker from resume_etf where market = `$"Developed"
select from historical where ticker in a[`ticker]

select distinct equity_class from resume_etf
select distinct region from resume_etf

//maximo aumento
select from historical where daily_retns=(max; daily_retns) fby ticker